In [1]:
from include.road_process import Road
from include.cool_space import CoolSpace
from include.building import Building
import geopandas as gpd
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import glob
import os

In [2]:
# set filepaths
directory_mac = "/Volumes/T7 Shield/TUD/Synthesis/cool_place/"
directory_win = "G:\\TUD\\Synthesis\\cool_place\\"

landuse_file = directory_win + "ams_landuse_top10NL.shp"
road_file = directory_win + "ams_roads_top10NL.shp"
building_file = directory_win + "ams_buildings_bagplus.shp"
shadow_path = directory_win + "shademaps\\"

In [3]:
# read road data
road = Road(gpd.read_file(road_file))

In [4]:
# read building data
building = Building(gpd.read_file(building_file))

In [5]:
# read land-use data
coolSpace = CoolSpace(gpd.read_file(landuse_file))

In [6]:
# read all shade maps from directory
shadows = []
shadow_files = glob.glob(os.path.join(shadow_path, '*.TIF'))
for shadow_file in shadow_files:
    shadow_map = rasterio.open(shadow_file, crs=coolSpace.data.crs)
    shadows.append(shadow_map)

shadows = shadows[0:2]  # use two shade maps for testing

In [7]:
# create road buffer
road.create_attribute('typeweg', 'buffer')
road.create_buffer('buffer')
road.data.set_geometry("buffered", inplace=True)

Buffered geometry column created.


In [8]:
coolSpace.clip(road.data, use_clip=False)

d:\softwares\anaconda3\envs\synthesis\Lib\site-packages\shapely\set_operations.py:76: RuntimeWarning: divide by zero encountered in difference
  return lib.difference(a, b, **kwargs)


In [9]:
# create building buffer and use it to clip the clipped land-use data
# the output will be the initial public space data
building.create_buffer(4)
building.data.set_geometry("buffered", inplace=True)
coolSpace.clip(building.data, use_clip=True)

Buffered geometry column created.


In [10]:
# perform shade calculation on public space data
coolSpace.calculate_shade(shadows, use_clip=True)
coolSpace.evaluate_shade_coverage()

Processing raster 1/2
POLYGON ((136019.5 476230.5, 136019.5 493749.5, 110000.5 493749.5, 110000.5 476230.5, 136019.5 476230.5))
Processing raster 2/2
POLYGON ((136019.5 476230.5, 136019.5 493749.5, 110000.5 493749.5, 110000.5 476230.5, 136019.5 476230.5))


D:\Files\GitHub\shady_amsterdam\cool_place\include\cool_space.py:269: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.data["tol_shade_avg"] += self.data[shade_avg_col].fillna(1)


In [11]:
# before exporting, transform the attributes contains lists into strings, so that
# it can be compatible with shapefile or geo-package format.
for i in range(len(shadows)):
    coolSpace.data[f"sdArea{i}"] = coolSpace.data[f"sdArea{i}"].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else str(x))

In [12]:
# export all shade geometries
for i in range(len(shadows)):
    shadeGeom = coolSpace.get_shade_geometries(i)
    shadeGeom.to_file(directory_win + "shadeGeoms.gpkg", layer=f"sdGeom{i}", driver="GPKG")

In [13]:
# export public space geometries
coolSpace_clipped = coolSpace.data.set_geometry("clipped", crs=coolSpace.data.crs)
coolSpace_clipped.drop(columns=["geometry"], inplace=True)
for i in range(len(shadows)):
    coolSpace_clipped.drop(columns=[f"sdGeom{i}"], inplace=True)
coolSpace_clipped.to_file(directory_win + "public_space.shp")

In [14]:
# export the original land-use geometries
coolSpace_origin = coolSpace.data.set_geometry("geometry")
coolSpace_origin.drop(columns=["clipped"], inplace=True)
for i in range(len(shadows)):
    coolSpace_origin.drop(columns=[f"sdGeom{i}"], inplace=True)
coolSpace_origin.to_file(directory_win + "land_use.shp")

In [15]:
# export cool space geometries
coolGeom = coolSpace.get_cool_spaces()
coolGeom.to_file(directory_win + "coolSpace.shp")

In [16]:
coolGeom_clipped = coolSpace.get_cool_spaces(geom_type='clipped')
coolGeom_clipped.to_file(directory_win + "coolSpace_clipped.shp")